In [16]:
from batching import get_selected_links, get_level_tags, get_level_one_tags, batch_cycle, get_batch_responses, check_batch_status, batch_get_relevant_content
import pandas as pd
from nslookup import Nslookup
import re
from googlesearch import search
import json


In [17]:
un_tags = pd.read_csv('./dados/tags/unspsc+.csv')
buyers = pd.read_csv('./dados/buyers/companies_with_summaries.csv')

In [18]:
buyers = buyers[~buyers['resumo_final'].isna()]
buyers = buyers.drop_duplicates(subset='ticker')

In [19]:
# level1_tags_response = get_level_one_tags(buyers, un_tags, id_column=2)
level1_tags_response = get_batch_responses(check_batch_status('batch_68b0e47a1f188190af0b2bd1c8b01e5e'))

[29/08/25 18:51:43] Batch Id: batch_68b0e47a1f188190af0b2bd1c8b01e5e,  Status: completed, Elapsed time 487917:51:43	


In [20]:
t = pd.DataFrame(level1_tags_response)
t['tag1'] = t['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['tags'])

In [21]:
buyers = buyers.merge(t[['custom_id', 'tag1']], left_on='ticker', right_on='custom_id', how='left').drop(columns=['custom_id'])

In [22]:
all_tags = {}
for resp in t[['custom_id','tag1']].itertuples(index=False):
    company_id = resp.custom_id
    tag1_values = resp.tag1
    all_tags[company_id] = {1: tag1_values}


In [8]:
level2_tags_response = get_level_tags(buyers, un_tags, 2, all_tags)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:06<00:00, 45.35it/s]


[29/08/25 07:37:28] Batch Id: batch_68b1821a77108190aed279f0e5cbf424,  Status: completed, Elapsed time 00:03:25			


In [10]:
t2 = pd.DataFrame(level2_tags_response)
t2['tag2'] = t2['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['tags'])

In [12]:
buyers = buyers.merge(t2[['custom_id', 'tag2']], left_on='ticker', right_on='custom_id', how='left').drop(columns=['custom_id'])

In [16]:
for resp in buyers[['ticker','tag1', 'tag2']].itertuples(index=False):
    company_id = resp.ticker
    tag1_values = resp.tag1
    tag2_values = resp.tag2
    all_tags[company_id] = {1: tag1_values, 2: tag2_values}

In [19]:
level3_tags_response = get_level_tags(buyers, un_tags, 3, all_tags)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:08<00:00, 33.54it/s]


[29/08/25 07:51:48] Batch Id: batch_68b184fc4a8c8190bf8ab8688ea1b14a,  Status: completed, Elapsed time 00:05:28			


In [20]:
t3 = pd.DataFrame(level3_tags_response)
t3['tag3'] = t3['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['tags'])

In [21]:
buyers = buyers.merge(t3[['custom_id', 'tag3']], left_on='ticker', right_on='custom_id', how='left').drop(columns=['custom_id'])

In [23]:
for resp in buyers[['ticker','tag1', 'tag2', 'tag3']].itertuples(index=False):
    company_id = resp.ticker
    tag1_values = resp.tag1
    tag2_values = resp.tag2
    tag3_values = resp.tag3
    all_tags[company_id] = {1: tag1_values, 2: tag2_values, 3: tag3_values}

In [24]:
level4_tags_response = get_level_tags(buyers, un_tags, 4, all_tags)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:13<00:00, 22.66it/s]


[29/08/25 08:32:52] Batch Id: batch_68b186dbb6188190a0400b3b16a08f11,  Status: completed, Elapsed time 00:38:32			


In [25]:
t4 = pd.DataFrame(level4_tags_response)
t4['tag4'] = t4['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['tags'])

In [26]:
buyers = buyers.merge(t4[['custom_id', 'tag4']], left_on='ticker', right_on='custom_id', how='left').drop(columns=['custom_id'])

In [30]:
for resp in buyers[['ticker','tag1', 'tag2', 'tag3', 'tag4']].itertuples(index=False):
    company_id = resp.ticker
    tag1_values = resp.tag1
    tag2_values = resp.tag2
    tag3_values = resp.tag3
    tag4_values = resp.tag4
    all_tags[company_id] = {1: tag1_values, 2: tag2_values, 3: tag3_values, 4: tag4_values}

In [31]:
level5_tags_response = get_level_tags(buyers, un_tags, 5, all_tags)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297/297 [00:16<00:00, 18.19it/s]


[29/08/25 08:53:09] Batch Id: batch_68b190783e088190b2abaff13bfad2b6,  Status: completed, Elapsed time 00:17:49			


In [32]:
t5 = pd.DataFrame(level5_tags_response)
t5['tag5'] = t5['response'].apply(lambda x: json.loads(x['body']['choices'][0]['message']['content'])['tags'])

In [33]:
buyers = buyers.merge(t5[['custom_id', 'tag5']], left_on='ticker', right_on='custom_id', how='left').drop(columns=['custom_id'])

In [34]:
buyers.to_csv('./dados/buyers/buyers_with_tags.csv', index=False)

In [101]:
buyers2 = pd.read_csv('./dados/buyers/buyers_with_tags.csv')

In [114]:
buyers['tag1'].iloc[1]

['Financials']

In [123]:
def df_to_codes_dict2(df: pd.DataFrame, tags_df: pd.DataFrame, verbose: bool = False):
    tags = {}
    depth = tags_df.shape[1]
    for row in df.iterrows():
        for level in range(1, depth):
            selected_tags = row[1][f'tag{level}']
            for selected_tag in selected_tags:
                try:
                    code_val = tags_df[tags_df[f'tag{level}'] == selected_tag]['code'].iloc[0]
                    tags[code_val] = selected_tag
                except IndexError:
                    if verbose:
                        print(f'{selected_tag} not found at level {level}')
                except ValueError:
                    if verbose:
                        print(f'{selected_tag} has invalid code at level {level}')
    return tags

In [124]:
from utils import remove_ancestors


buyers['codes_tags'] = buyers.apply(lambda x: list(remove_ancestors(df_to_codes_dict2(x.to_frame().T, un_tags)).keys()), axis=1)

In [127]:
buyers[['Nome','ticker','valor','resumo_final','codes_tags']].to_parquet('./dados/buyers/buyers_final.parquet')

In [3]:
final_buyers = pd.read_parquet('./dados/buyers/buyers_final.parquet')

In [9]:
companies_financials = pd.read_csv('./dados/buyers/companies_financials.csv')
ebitdas = companies_financials[(companies_financials['nome_coluna']=='EBITDA') & (companies_financials['data']=='2024')]
ebitdas = ebitdas.rename(columns={'valor':'ebitda_2024'})
dl = companies_financials[(companies_financials['data']=='2024')&(companies_financials['nome_coluna']==('Dívida líquida'))]
dl = dl.rename(columns={'valor':'dl_2024'})

In [10]:
final_buyers.merge(
    ebitdas[['ticker', 'ebitda_2024']],
    left_on='ticker',
    right_on='ticker',
    how='left'
).merge(
    dl[['ticker', 'dl_2024']],
    left_on='ticker',
    right_on='ticker',
    how='left'
)

,Nome,ticker,valor,resumo_final,codes_tags,ebitda_2024,dl_2024
0,BRF,brfs3,https://www.brf-global.com,BRF is one of the largest food companies globa...,"[12.50.19, 12.50.11.21, 12.50.11.22, 12.50.11....",8.846000e+09,8.681000e+09
1,B3,b3sa3,https://www.b3.com.br,B3 is a Brazilian company that operates as one...,"[14.64.11.15, 14.64.11.17, 14.84.12.18.01, 14....",6.617000e+09,-7.200000e+07
2,Cogna,cogn3,https://www.cogna.com.br,Cogna Educação is a leading Brazilian educatio...,"[11.86.13.20, 11.86.10.17, 11.86.11.16.02, 11....",1.809000e+09,5.541000e+09
3,Lojas Renner,lren3,https://www.lojasrennersa.com.br,Lojas Renner S.A. is the largest fashion and l...,"[11.53.11, 11.56.14, 11.53.10.18, 11.53.10.17,...",2.503000e+09,-1.826000e+09
4,Banco Bradesco,bbdc4,https://www.bradesco.com.br,Bradesco is a financial services company based...,"[14.84.12.15.01, 14.84.13.15.02, 14.84.13.15.0...",0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
337,Iguatemi,igti4,https://www.jereissati.com.br,Iguatemi S.A. is a Brazilian company specializ...,"[11.90.11, 19.95.12.21, 11.90.10.15, 19.95.12....",9.140000e+08,1.627000e+09
338,Iguatemi,igti4,https://www.jereissati.com.br,Iguatemi S.A. is a Brazilian company specializ...,"[11.90.11, 19.95.12.21, 11.90.10.15, 19.95.12....",9.140000e+08,1.627000e+09
339,Iguatemi,igti4,https://www.jereissati.com.br,Iguatemi S.A. is a Brazilian company specializ...,"[11.90.11, 19.95.12.21, 11.90.10.15, 19.95.12....",9.140000e+08,1.627000e+09
340,Iguatemi,igti4,https://www.jereissati.com.br,Iguatemi S.A. is a Brazilian company specializ...,"[11.90.11, 19.95.12.21, 11.90.10.15, 19.95.12....",9.140000e+08,1.627000e+09


In [15]:
from utils import build_codes_df


build_codes_df(['12.50.19', '12.50.11.21', '12.50.11.22', '12.50.11.20.11', '12.50.11.20.09',
 '12.50.11.20.13', '12.50.11.20.21', '12.50.11.20.23'], un_tags)

,tag1,tag2,tag3,tag4,tag5
0,Consumer Staples,Food and Beverage Products,Meat and poultry products,Non gmo processed meat and poultry products,
1,Consumer Staples,Food and Beverage Products,Meat and poultry products,Organic processed meat and poultry products,
2,Consumer Staples,Food and Beverage Products,Meat and poultry products,Processed meat and poultry products,"Chicken, processed with additions"
3,Consumer Staples,Food and Beverage Products,Meat and poultry products,Processed meat and poultry products,"Pork, processed with additions"
4,Consumer Staples,Food and Beverage Products,Meat and poultry products,Processed meat and poultry products,"Turkey, processed with additions"
5,Consumer Staples,Food and Beverage Products,Meat and poultry products,Processed meat and poultry products,"Specialty poultry, processed with additions"
6,Consumer Staples,Food and Beverage Products,Meat and poultry products,Processed meat and poultry products,"Mixed species meat, processed with additions"
7,Consumer Staples,Food and Beverage Products,Prepared and preserved foods,,
